# 월간식당 빅데이터 분석

* 키워드별 단어 출현 빈도

In [1]:
import itertools
import json
import numpy as np
import pandas as pd
import random 
import re
import string
import warnings
warnings.simplefilter("ignore")

from collections import Counter
from gensim.models.word2vec import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from konlpy.utils import pprint
from konlpy.tag import Hannanum
hannanum = Hannanum()
from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import Kkma
kkma = Kkma()
import matplotlib.pyplot as plt
%matplotlib inline

from pandas import read_excel
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
from tqdm import trange
from wordcloud import WordCloud

from msba import posts as p
from msba import stopwords as stopwords

In [2]:
def interested_words():
    # 핵심단어 읽어 오기
    my_sheet = '소비키워드'
    keywords_filename = 'deskresearch_.xlsx'
    df = read_excel(keywords_filename, sheet_name = my_sheet, header=1) # index_col='번호'
    keywords = df['핵심단어']
    subkeywords = df['대체어']
    interested_words = df['키워드']
    return keywords, subkeywords, interested_words

def oneDArray(x):
    return list(itertools.chain(*x))

def preprocessing(text):
    # 개행문자 제거
    text = re.sub('\\\\n', ' ', text)
#     text = re.sub('[A-z]', '', text)
#     text = re.sub('[0-9]', '', text)
    text = re.sub('[\'\"]', '', text)
    text = re.sub('[\'\n"]', '', text)
    res = ''.join([i for i in text if not i.isdigit()]) 
    return text

def preprocessing_2(rows):
    pattern = re.compile(r"[.,?!★~]")
    sentences = []
    for row in rows:
        sentences = sentences + pattern.split(row.replace("\xa0", "").replace("\t","").strip())
        sentences = list(set(sentences))        
    return sentences

def convert_list_to_string(org_list, seperator=' '):
    """ Convert list to string, by joining all item in list with given separator.
        Returns the concatenated string """
    return seperator.join(org_list)

# 실전 : 말뭉치 생성 및 전처리
def preprocessing_3(sentences):
    # Convert list of strings to string
#     full_str = convert_list_to_string(sentences[1])
#     # print(full_str)

    pattern = re.compile(r".*(광고정보).*")

    corpus = []
    for sentence in sentences:
        if len(sentence):
            sentence = sentence.strip().split(" ")
            sentence = convert_list_to_string(sentence).replace("  ", " ").strip()
            if pattern.match(sentence):
                continue
            else:
                corpus.append(sentence)
    return corpus

def preprocessing_4(docs):
    # docs = [
    #         w for w in hannanum.nouns(" ".join(cell)) for cell in cells
    #         if ((not w[0].isnumeric()) and (w[0] not in string.punctuation))
    # ]
    vect = CountVectorizer(stop_words=stopwords.stopwords_kr, min_df=5, max_df=200).fit(docs)
    count = vect.transform(docs).toarray().sum(axis=0)
    idx = np.argsort(-count)
    count = count[idx]
    feature_name = np.array(vect.get_feature_names())[idx]
    # plt.bar(range(len(count)), count)
    # plt.show()

    tf_list = list(zip(feature_name, count))[:100]

    tf_df = pd.DataFrame(tf_list,columns=['단어', '빈도'])
    return tf_df

def remove_stopwords_from_list(lst):
    result = []
    for w in lst:
        if ((len(w) > 1) and (not w.isdigit())):
            if w not in stopwords.stopwords_kr: 
                result.append(w)
    return result

def remove_stopwords(line):
    result = ""
    for w in line.split(" "):
        if ((len(w) > 1) and (not w.isdigit())):
            if w not in stopwords.stopwords_kr: 
                result = result + w + " "
    return result.strip()

def save_to_csv(keyword, output, tf_df):
    # save to csv    
    filename = "./output/" + output + "_" + keyword.replace(" ","") + ".csv"   
    # filename_list.append(filename)
    tf_df.to_csv(filename, date_format='%Y%m%d', encoding='utf-8-sig')

## 키워드/관심어 불러오기

In [3]:
keywords, subkeywords, interested_words = interested_words()

for keyword, subkeyword, interested_word in zip(keywords, subkeywords, interested_words):
    subkeyword = subkeyword.replace(" ", "").replace(",","|")
    interested_word = subkeyword + "|" + interested_word.replace(" ", "").replace(",","|")
#     print(keyword, " : " , subkeyword, interested_word)

# 자신의 관심사에 맞는 단어로 데이터 가져오기

In [14]:
# %%time
for keyword, subkeyword, interested_word in tqdm(zip(keywords, subkeywords, interested_words)):

    #     keyword = '1인 외식'
    # keyword = keyword.replace(" ","")
    df = p.readall(keyword.replace(" ",""))
    df = df[ (df['date'] >= '2019-07-01') & (df['date'] < '2020-07-01')]
    df = df.drop_duplicates()
    # print(df.shape)  
    rows = df['title'].apply(preprocessing) + df['content'].apply(preprocessing)
    # print(len(rows), type(rows))
    sentences = preprocessing_2(rows)
    # print(len(sentences), type(sentences))
    sentences = preprocessing_3(sentences)
    # print(len(corpus), type(corpus))

    # 4. 각 문장별로 형태소 구분하기
    sentences_tag = []
    for sentence in sentences:
        morph = okt.pos(sentence)
        sentences_tag.append(morph)
    #     print(morph)
    #     print('-'*30)

    # print(sentences_tag)
    # print(len(sentences_tag))
    # print('\n'*3)

#     sentences_tag

    # 5. 명사 혹은 형용사인 품사만 선별해 리스트에 담기
    noun_adj_list = []

    noun_adj_list = []
    for sentence1 in sentences_tag:
        words = ""
        for word, tag in sentence1:
            if tag in ['Noun','Adjective']:
                words = words + word + " "
    #     print(words)
    #     print("------------")
        words = remove_stopwords(words)
        noun_adj_list.append(words)

    #             line = remove_stopwords(line)
    #         print(line)
    #     noun_adj_list.append(line)

#     noun_adj_list

    #########################################################################
    # 최빈어
    #########################################################################
    # import itertools

    # def oneDArray(x):
    #     return list(itertools.chain(*x))

    tf_list = []
    for sentence1 in noun_adj_list:
        sentence1.split()
        tf_list.append(sentence1.split())

#     type(tf_list)

    tf_list = oneDArray(tf_list)
    
#     tf_list

    # 6. 선별된 품사별 빈도수 계산 & 상위 빈도 10위 까지 출력
    counts = Counter(tf_list)

    toplist = counts.most_common(200)

    # type(toplist)

    tf_df = pd.DataFrame (toplist,columns=['단어', '빈도수'])
    save_to_csv(keyword, "최빈어", tf_df)
    
   
    #########################################################################
    # 유사도
    #########################################################################
    # using remove() to 
    # perform removal 
    while("" in noun_adj_list) : 
        noun_adj_list.remove("") 

    dataset = pd.DataFrame(noun_adj_list, columns=['문장'])

    nan_value = float("NaN")
    dataset.replace("", nan_value, inplace=True)
    dataset.dropna(subset = ["문장"], inplace=True)
    # dataset.reindex

    # noun_adj_list[12]
    # dataset['문장'][13]

    tmp_corpus = dataset['문장'].map(lambda x: x.split('.'))
    # tmp_corpus

    # type(tmp_corpus)
    # tmp_corpus[11]
    # tmp_corpus[13]

    # corpus [[w1,w2,w3..],[..]]
    corpus = []
    for i in range(len(tmp_corpus)):
        for line in tmp_corpus[i]:
    #         print(i)
            words = [x for x in line.split()]
            corpus.append(words)

#     corpus

    num_of_sentences = len(corpus)
    num_of_words = 0
    for line in corpus:
        num_of_words += len(line)

    print('Num of sentences - %s'%(num_of_sentences))
    print('Num of words - %s'%(num_of_words))

    phrases = Phrases(sentences=corpus,min_count=25,threshold=50)
    bigram = Phraser(phrases)

    for index,sentence in enumerate(corpus):
        corpus[index] = bigram[sentence]

    # shuffle corpus
    def shuffle_corpus(sentences):
        shuffled = list(sentences)
        random.shuffle(shuffled)
        return shuffled

    # sg - skip gram |  window = size of the window | size = vector dimension
    size = 100
    window_size = 5 # sentences weren't too long, so
    epochs = 100
    min_count = 2
    workers = 4

    # train word2vec model using gensim
    # model = Word2Vec(corpus, sg=1,window=window_size,size=size,
    #                  min_count=min_count,workers=workers,iter=epochs,sample=0.01)

#     %%time
    model = Word2Vec(dataset,            # 불용어 처리한 후 
                    sg=2,                # skip-gram 적용: 중심 단어로 주변 단어를 예측
                    window=window_size,  # 중심 단어로부터 좌우 5개 단어까지 학습에 적용
                    iter=epochs,
                    #workers=workers,
                    #size=size,
                    sample=0.01,
                    min_count=1          # 전체문서에서 최소 1회 이상 출현단어로 학습 진행             
                    )
    model.init_sims(replace=True)

    model.build_vocab(sentences=shuffle_corpus(corpus),update=True)

    for i in trange(5):
        model.train(sentences=shuffle_corpus(corpus),epochs=50,total_examples=model.corpus_count)

#     corpus

    # save model
    modelname = "./model/" + "w2v_" + keyword.replace(" ","")  
    model.save(modelname)

0it [00:00, ?it/s]

Num of sentences - 35366
Num of words - 200682



100%|██████████| 5/5 [02:00<00:00, 24.19s/it]
1it [04:28, 268.75s/it]

Num of sentences - 41273
Num of words - 247511



100%|██████████| 5/5 [02:35<00:00, 31.18s/it]
2it [09:49, 284.38s/it]

...


100%|██████████| 5/5 [01:17<00:00, 15.59s/it]
33it [2:16:11, 344.63s/it]

Num of sentences - 19630
Num of words - 105647



100%|██████████| 5/5 [00:56<00:00, 11.20s/it]
34it [2:18:16, 278.78s/it]

Num of sentences - 20445
Num of words - 123214



100%|██████████| 5/5 [01:10<00:00, 14.10s/it]
35it [2:20:44, 241.28s/it]
